In [1]:
from glob import glob
import pandas as pd
import numpy as np
from Bio import PDB
import os
os.environ["PATH"] += os.pathsep + "/opt/conda/envs/team05/bin"
from rdkit import Chem
from rdkit.Chem import MolToSmiles, MolFromMol2File
from rdkit.Chem.rdmolfiles import MolFromSmiles, MolToSmiles
from Bio.PDB import Select, PDBIO
from Bio.PDB.PDBParser import PDBParser

In [2]:
pdb_path = os.path.abspath("datasets/PDBbind/general-set")
info_path = os.path.abspath("datasets/")

In [3]:
complex_list = [f for f in os.listdir(pdb_path) if not f.startswith(".")]

In [4]:
def strip_hetatm(src_file: str, dst_file: str):
    """PDB 텍스트를 스트리밍으로 읽어 HETATM을 제거하고 저장"""
    # 필요하다면 디렉토리 생성
    os.makedirs(os.path.dirname(dst_file), exist_ok=True)
    with open(src_file, "r") as fin, open(dst_file, "w") as fout:
        for line in fin:
            rec = line[:6]
            # ATOM/TER/END/MODEL/ENDMDL 같은 레코드만 저장 (HETATM은 건너뛰기)
            if rec == "ATOM  " or rec == "TER   " or rec == "END   " or rec == "MODEL " or rec == "ENDMDL":
                fout.write(line)

def remove_HETATM_PDBbind(pdb_ids):
    for pid in pdb_ids:
        src = f"{pdb_path}/{pid}/{pid}_protein.pdb"
        dst = f"{pdb_path}/{pid}/{pid}_remove_HETATM_protein.pdb"
        if not os.path.isfile(src):
            # 없는 파일은 건너뜀
            continue
        strip_hetatm(src, dst)

In [5]:
remove_HETATM_PDBbind(complex_list)

In [6]:
pdb_parser = PDB.PDBParser(QUIET = True)

In [7]:
amino_acids_short = {
    "ALA":"A", "ARG":"R", "ASN":"N", "ASP":"D", "CYS":"C",
    "GLU":"E", "GLN":"Q", "GLY":"G", "HIS":"H", "ILE":"I",
    "LEU":"L", "LYS":"K", "MET":"M", "PHE":"F", "PRO":"P",
    "SER":"S", "THR":"T", "TRP":"W", "TYR":"Y", "VAL":"V",
    "SEC":"U", "PYL":"O"
}

In [8]:
data_df = pd.DataFrame({"PDB":complex_list})

In [9]:
def get_info(pdb):
    try:
        """ Load protein info """
        structure = pdb_parser.get_structure(pdb, f"{pdb_path}/{pdb}/{pdb}_remove_HETATM_protein.pdb")
        chain_name_list, pdb_sequence_list, seq_lengths_list, protein_atom_coords, protein_atom_residue_list, reindex = list(), list(), list(), list(), list(), 0
        
        """ Extract protein info """
        for chain_name in list(structure[0].child_dict.keys()):
            chain = structure[0][chain_name]

            pdb_sequence = ""
            for residue in chain.get_residues():
                if residue.resname in amino_acids_short.keys():
                    pdb_sequence += amino_acids_short[residue.resname]

                    for atom in residue:
                        protein_atom_coords.append(atom.get_coord())
                        protein_atom_residue_list.append(reindex)
                    reindex += 1     

            if len(pdb_sequence) != 0:
                chain_name_list.append(chain_name)
                pdb_sequence_list.append(pdb_sequence)
                seq_lengths_list.append(len(pdb_sequence))

        """ Load pocket info """      
        protein_atom_coords, binding_index = np.array(protein_atom_coords), list()
        pocket_structure = pdb_parser.get_structure(pdb, f"{pdb_path}/{pdb}/{pdb}_pocket.pdb")

        pocket_coordi = list()

        for chain_name in list(pocket_structure[0].child_dict.keys()):
            chain = pocket_structure[0][chain_name]
            for residue in chain.get_residues():
                if residue.resname in amino_acids_short.keys():
                    for atom in residue:
                        pocket_coordi.append(atom.get_coord())
        
        """ Matching pocket info """
        pocket_coordi = np.array(pocket_coordi)
        bi_x, bi_y, bi_z = pocket_coordi[:, 0], pocket_coordi[:, 1], pocket_coordi[:, 2]

        for i, j, k in zip(bi_x, bi_y, bi_z):
            tmp_coordi = np.array([i, j, k], dtype = np.float32)
            ind = np.where((protein_atom_coords == tmp_coordi).all(axis = 1))[0][0]
            binding_index.append(protein_atom_residue_list[ind])

        binding_index = sorted(list(set(binding_index)))
        binding_index = list(map(str, binding_index))

        total_seq_lengths = np.sum(np.array(seq_lengths_list))
        seq_lengths_list = list(map(str, seq_lengths_list))

        return ",".join(chain_name_list), ",".join(pdb_sequence_list), total_seq_lengths, ",".join(seq_lengths_list), ",".join(binding_index)

    except Exception as e:
        print(pdb, e)
        return None

In [10]:
from multiprocessing import Process, Queue, Pool

def parallelize_dataframe(df, func, num_partitions=5):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_partitions)
    results = pool.map(func, df_split)
    pool.close()
    pool.join()
    return results

In [11]:
def get_pdb_info_bulk(df):
    return df.PDB.map(get_info)

In [12]:
info_results = parallelize_dataframe(data_df, get_pdb_info_bulk, 5)

/opt/conda/envs/team05/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


readme [Errno 2] No such file or directory: '/workspace/datasets/PDBbind/general-set/readme/readme_remove_HETATM_protein.pdb'
index [Errno 2] No such file or directory: '/workspace/datasets/PDBbind/general-set/index/index_remove_HETATM_protein.pdb'


In [13]:
info_results = pd.concat(info_results)

In [14]:
data_df["Chain"] = info_results.map(lambda a: a[0] if a is not None else None)

In [15]:
data_df["Sequence"] = info_results.map(lambda a: a[1] if a is not None else None)

In [16]:
data_df["Total_seq_lengths"] = info_results.map(lambda a: a[2] if a is not None else None)

In [17]:
data_df["Chain_seq_lengths"] = info_results.map(lambda a: a[3] if a is not None else None)

In [18]:
data_df["BS"] = info_results.map(lambda a: a[4] if a is not None else None)

In [19]:
data_df = data_df.loc[data_df.Sequence.isna()==False].reset_index(drop=True)
data_df = data_df.loc[data_df.Chain != " "].reset_index(drop=True)

In [20]:
data_df

,PDB,Chain,Sequence,Total_seq_lengths,Chain_seq_lengths,BS
0,5svl,"A,C",KSVVVKSWTIGIINRVVQLLIISYFVGWVFLHEKAYQVRDTAIESS...,666.0,"333,333","81,83,97,100,113,114,115,142,144,221,255,258,2..."
1,5ali,A,TLRAAVFDLDGVLALPAVFGVLGRTEEALALPRGLLNDAFQKGGPE...,546.0,546,"264,265,266,267,308,332,333,334,335,336,337,33..."
2,5a5d,A,AMLPISMSDEGDSFLVKDSLGENKIPKNPSKVVILDLGILDTFDAL...,289.0,289,"57,58,74,75,76,77,94,95,96,97,98,99,100,117,16..."
3,6bh5,A,EFVPPPECPVFEPSWEEFTDPLSFIGRIRPLAEKTGICKIRPPKDW...,292.0,292,"61,63,123,124,125,126,127,173,174,175,176,177,..."
4,1hqh,"A,B,C",KPIEIIGAPFSKGQPRGGVEKGPAALRKAGLVEKLKETEYNVRDHG...,942.0,"314,314,314","12,13,14,15,18,62,94,95,96,116,118,119,120,121..."
...,...,...,...,...,...,...
14117,6g6y,A,KNIQVVVRCRPFNLAERKASAHSIVECDPVRKEVSVRTGSSRKTYT...,319.0,319,"56,90,91,92,93,94,95,96,97,98,103,104,105,106,..."
14118,3wc7,A,GHSYEKYNNWETIEAWTKQVTSENPDLISRTAIGTTFLGNNIYLLK...,304.0,304,"62,63,65,66,69,119,121,136,137,138,139,140,143..."
14119,5zhm,"A,B",SMLWVGVVSIFPEMFRAISDYGITSRAVKQGLLTLTCWNPRVYTED...,476.0,"239,237","59,60,61,62,63,88,89,90,91,92,93,95,96,113,114..."
14120,3fts,A,VDTCSLASPASVCRTKHLHLRCSVDFTRRTLTGTAALTVQSQEDNL...,607.0,607,"3,130,131,132,133,134,135,146,196,262,263,266,..."


In [21]:
lengths = data_df.Total_seq_lengths.values

In [22]:
data_df = data_df[lengths <= 1500].reset_index(drop=True)
data_df

,PDB,Chain,Sequence,Total_seq_lengths,Chain_seq_lengths,BS
0,5svl,"A,C",KSVVVKSWTIGIINRVVQLLIISYFVGWVFLHEKAYQVRDTAIESS...,666.0,"333,333","81,83,97,100,113,114,115,142,144,221,255,258,2..."
1,5ali,A,TLRAAVFDLDGVLALPAVFGVLGRTEEALALPRGLLNDAFQKGGPE...,546.0,546,"264,265,266,267,308,332,333,334,335,336,337,33..."
2,5a5d,A,AMLPISMSDEGDSFLVKDSLGENKIPKNPSKVVILDLGILDTFDAL...,289.0,289,"57,58,74,75,76,77,94,95,96,97,98,99,100,117,16..."
3,6bh5,A,EFVPPPECPVFEPSWEEFTDPLSFIGRIRPLAEKTGICKIRPPKDW...,292.0,292,"61,63,123,124,125,126,127,173,174,175,176,177,..."
4,1hqh,"A,B,C",KPIEIIGAPFSKGQPRGGVEKGPAALRKAGLVEKLKETEYNVRDHG...,942.0,"314,314,314","12,13,14,15,18,62,94,95,96,116,118,119,120,121..."
...,...,...,...,...,...,...
13714,6g6y,A,KNIQVVVRCRPFNLAERKASAHSIVECDPVRKEVSVRTGSSRKTYT...,319.0,319,"56,90,91,92,93,94,95,96,97,98,103,104,105,106,..."
13715,3wc7,A,GHSYEKYNNWETIEAWTKQVTSENPDLISRTAIGTTFLGNNIYLLK...,304.0,304,"62,63,65,66,69,119,121,136,137,138,139,140,143..."
13716,5zhm,"A,B",SMLWVGVVSIFPEMFRAISDYGITSRAVKQGLLTLTCWNPRVYTED...,476.0,"239,237","59,60,61,62,63,88,89,90,91,92,93,95,96,113,114..."
13717,3fts,A,VDTCSLASPASVCRTKHLHLRCSVDFTRRTLTGTAALTVQSQEDNL...,607.0,607,"3,130,131,132,133,134,135,146,196,262,263,266,..."


In [23]:
def convert_smiles(row):
    pdb = row.PDB
    
    mol = f"{pdb_path}/{pdb}/{pdb}_ligand.mol2"
    command = f"obabel -imol2 {mol} -osmi -xC | obabel -ismi -osmi -xk -O tmp.smi"
    os.system(command)
    
    smiles = read_file(open("tmp.smi"))[0].split('\t')[0].strip()
    
    try:
        smiles = MolToSmiles(MolFromSmiles(smiles),isomericSmiles = False, kekuleSmiles = True)
        return smiles
    
    except Exception as e:
        print(pdb, e)
        return None

In [24]:
def read_file(file):
    return [i.strip() for i in file.readlines()]

In [25]:
SMILES = data_df.apply(convert_smiles, axis = 1)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 5svl_ligand)

1 molecule converted
*** Open Babel Warning  in ParseSmiles
  Failed to kekulize aromatic SMILES (title is 5svl_ligand)

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 1hqh_ligand)

1 molecule converted
*** Open Babel Warning  in ParseSmiles
  Failed to kekulize aromatic SMILES (title is 1hqh_ligand)

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 5w8v_ligand)


4wkt Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(RDKit::ROMol mol, RDKit::SmilesWriteParams params)


1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 2p0x_ligand)

1 molecule converted
*** Open Babel Warning  in ParseSmiles
  Failed to kekulize aromatic SMILES (title is 2p0x_ligand)

1 molecule converted
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 4j8s_ligand)

1 molecule converted
*** Open Babel Warning  in ParseSmiles
  Failed to kekulize aromatic SMILES (title is 4j8s_ligand)

1 molecule converted
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 6e83_ligand)

1 molecule converted
*** Open Babel Warning  in ParseSmiles
  Failed to kekulize aromatic SMILES (title is 6e83_ligand)

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 mo

3shj Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(RDKit::ROMol mol, RDKit::SmilesWriteParams params)


1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 3r0h_ligand)

1 molecule converted
*** Open Babel Warning  in ParseSmiles
  Failed to kekulize aromatic SMILES (title is 3r0h_ligand)

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
[14:47:45] Explicit valence for atom # 21 O, 3, is greater than permitted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted


...

1 molecule converted
1 molecule converted
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 2jq9_ligand)

1 molecule converted
*** Open Babel Warning  in ParseSmiles
  Failed to kekulize aromatic SMILES (title is 2jq9_ligand)

1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 3aav_ligand)

1 molecule converted
*** Open Babel Warning  in ParseSmiles
  Failed to kekulize aromatic SMILES (title is 3aav_ligand)

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 4azy_ligand)

1 molecule converted
*** Open Babel Warning  in ParseSmiles
  Failed to kekulize aromatic SMILES (title is 4azy_ligand)

1 mo

6buv Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(RDKit::ROMol mol, RDKit::SmilesWriteParams params)


*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 4ym4_ligand)

1 molecule converted
*** Open Babel Warning  in ParseSmiles
  Failed to kekulize aromatic SMILES (title is 4ym4_ligand)

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 5yji_ligand)

1 molecule converted
*** Open Babel Warning  in ParseSmiles
  Failed to kekulize aromatic SMILES (title is 5yji_ligand)

1 molecule converted
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 3mxy_ligand)

1 molecule converted
*** Open Babel Warning  in ParseSmiles
  Failed to kekulize aromatic SMILES (title is 3mxy_ligand)

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 mo

1sbr Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(RDKit::ROMol mol, RDKit::SmilesWriteParams params)


1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 3oka_ligand)

1 molecule converted
*** Open Babel Warning  in ParseSmiles
  Failed to kekulize aromatic SMILES (title is 3oka_ligand)

1 molecule converted
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 3ax5_ligand)

1 molecule converted
*** Open Babel Warning  in ParseSmiles
  Failed to kekulize aromatic SMILES (title is 3ax5_ligand)

1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 1lvc_ligand)

1 molecule converted
*** Open Babel Warning  in ParseSmiles
  Failed to kekulize aromatic SMILES (title is 1lvc_ligand)

1 mo

In [26]:
data_df["SMILES"] = SMILES

In [27]:
data_df = data_df.loc[data_df.SMILES.isna()==False].reset_index(drop=True)
data_df

,PDB,Chain,Sequence,Total_seq_lengths,Chain_seq_lengths,BS,SMILES
0,5svl,"A,C",KSVVVKSWTIGIINRVVQLLIISYFVGWVFLHEKAYQVRDTAIESS...,666.0,"333,333","81,83,97,100,113,114,115,142,144,221,255,258,2...",NC1=C2N=CN(C3OC(COP([O])(=O)OP([O])(=O)OP([O])...
1,5ali,A,TLRAAVFDLDGVLALPAVFGVLGRTEEALALPRGLLNDAFQKGGPE...,546.0,546,"264,265,266,267,308,332,333,334,335,336,337,33...",COC1=CC=C(NC(=O)NC2=CC=CC(C3=C(C4=NC5=C(N=CN=C...
2,5a5d,A,AMLPISMSDEGDSFLVKDSLGENKIPKNPSKVVILDLGILDTFDAL...,289.0,289,"57,58,74,75,76,77,94,95,96,97,98,99,100,117,16...",O=C(NCCCCCNC(=O)C1=CC=CC(O)=C1O)C1=CC=CC(O)=C1O
3,6bh5,A,EFVPPPECPVFEPSWEEFTDPLSFIGRIRPLAEKTGICKIRPPKDW...,292.0,292,"61,63,123,124,125,126,127,173,174,175,176,177,...",O=C([O-])C1=C2NC(C(OCCC[NH+]3CCCCC3)C3=CC=CC=C...
4,1hqh,"A,B,C",KPIEIIGAPFSKGQPRGGVEKGPAALRKAGLVEKLKETEYNVRDHG...,942.0,"314,314,314","12,13,14,15,18,62,94,95,96,116,118,119,120,121...",N[C](N=O)NCCC([NH3+])C(=O)[O-]
...,...,...,...,...,...,...,...
13471,6g6y,A,KNIQVVVRCRPFNLAERKASAHSIVECDPVRKEVSVRTGSSRKTYT...,319.0,319,"56,90,91,92,93,94,95,96,97,98,103,104,105,106,...",CC(=O)NC1[NH2+]N(C(C)=O)C(C)(C2=CC=CC=C2)S1
13472,3wc7,A,GHSYEKYNNWETIEAWTKQVTSENPDLISRTAIGTTFLGNNIYLLK...,304.0,304,"62,63,65,66,69,119,121,136,137,138,139,140,143...",CC(C)C(NC(=O)CCC1=CC=CC=C1)P([O])(=O)CC(CCCCC[...
13473,5zhm,"A,B",SMLWVGVVSIFPEMFRAISDYGITSRAVKQGLLTLTCWNPRVYTED...,476.0,"239,237","59,60,61,62,63,88,89,90,91,92,93,95,96,113,114...",CC[NH+](CC)CC1=CC=C(CNC(=O)C2=CSC3=C2C(=O)N=CN...
13474,3fts,A,VDTCSLASPASVCRTKHLHLRCSVDFTRRTLTGTAALTVQSQEDNL...,607.0,607,"3,130,131,132,133,134,135,146,196,262,263,266,...",OC1=CC=C(C=CC2=CC(O)=CC(O)=C2)C=C1


In [28]:
def get_SMILES_length(df):
    index = [True if len(smi) <= 160 else False for smi in df.SMILES.values]
    return index

In [29]:
smiles_index = get_SMILES_length(data_df)

In [30]:
data_df = data_df.loc[smiles_index].reset_index(drop=True)

In [31]:
data_df

,PDB,Chain,Sequence,Total_seq_lengths,Chain_seq_lengths,BS,SMILES
0,5svl,"A,C",KSVVVKSWTIGIINRVVQLLIISYFVGWVFLHEKAYQVRDTAIESS...,666.0,"333,333","81,83,97,100,113,114,115,142,144,221,255,258,2...",NC1=C2N=CN(C3OC(COP([O])(=O)OP([O])(=O)OP([O])...
1,5ali,A,TLRAAVFDLDGVLALPAVFGVLGRTEEALALPRGLLNDAFQKGGPE...,546.0,546,"264,265,266,267,308,332,333,334,335,336,337,33...",COC1=CC=C(NC(=O)NC2=CC=CC(C3=C(C4=NC5=C(N=CN=C...
2,5a5d,A,AMLPISMSDEGDSFLVKDSLGENKIPKNPSKVVILDLGILDTFDAL...,289.0,289,"57,58,74,75,76,77,94,95,96,97,98,99,100,117,16...",O=C(NCCCCCNC(=O)C1=CC=CC(O)=C1O)C1=CC=CC(O)=C1O
3,6bh5,A,EFVPPPECPVFEPSWEEFTDPLSFIGRIRPLAEKTGICKIRPPKDW...,292.0,292,"61,63,123,124,125,126,127,173,174,175,176,177,...",O=C([O-])C1=C2NC(C(OCCC[NH+]3CCCCC3)C3=CC=CC=C...
4,1hqh,"A,B,C",KPIEIIGAPFSKGQPRGGVEKGPAALRKAGLVEKLKETEYNVRDHG...,942.0,"314,314,314","12,13,14,15,18,62,94,95,96,116,118,119,120,121...",N[C](N=O)NCCC([NH3+])C(=O)[O-]
...,...,...,...,...,...,...,...
12473,6g6y,A,KNIQVVVRCRPFNLAERKASAHSIVECDPVRKEVSVRTGSSRKTYT...,319.0,319,"56,90,91,92,93,94,95,96,97,98,103,104,105,106,...",CC(=O)NC1[NH2+]N(C(C)=O)C(C)(C2=CC=CC=C2)S1
12474,3wc7,A,GHSYEKYNNWETIEAWTKQVTSENPDLISRTAIGTTFLGNNIYLLK...,304.0,304,"62,63,65,66,69,119,121,136,137,138,139,140,143...",CC(C)C(NC(=O)CCC1=CC=CC=C1)P([O])(=O)CC(CCCCC[...
12475,5zhm,"A,B",SMLWVGVVSIFPEMFRAISDYGITSRAVKQGLLTLTCWNPRVYTED...,476.0,"239,237","59,60,61,62,63,88,89,90,91,92,93,95,96,113,114...",CC[NH+](CC)CC1=CC=C(CNC(=O)C2=CSC3=C2C(=O)N=CN...
12476,3fts,A,VDTCSLASPASVCRTKHLHLRCSVDFTRRTLTGTAALTVQSQEDNL...,607.0,607,"3,130,131,132,133,134,135,146,196,262,263,266,...",OC1=CC=C(C=CC2=CC(O)=CC(O)=C2)C=C1


In [32]:
data_df = data_df.iloc[:, [0, 2, 5]]
data_df

,PDB,Sequence,BS
0,5svl,KSVVVKSWTIGIINRVVQLLIISYFVGWVFLHEKAYQVRDTAIESS...,"81,83,97,100,113,114,115,142,144,221,255,258,2..."
1,5ali,TLRAAVFDLDGVLALPAVFGVLGRTEEALALPRGLLNDAFQKGGPE...,"264,265,266,267,308,332,333,334,335,336,337,33..."
2,5a5d,AMLPISMSDEGDSFLVKDSLGENKIPKNPSKVVILDLGILDTFDAL...,"57,58,74,75,76,77,94,95,96,97,98,99,100,117,16..."
3,6bh5,EFVPPPECPVFEPSWEEFTDPLSFIGRIRPLAEKTGICKIRPPKDW...,"61,63,123,124,125,126,127,173,174,175,176,177,..."
4,1hqh,KPIEIIGAPFSKGQPRGGVEKGPAALRKAGLVEKLKETEYNVRDHG...,"12,13,14,15,18,62,94,95,96,116,118,119,120,121..."
...,...,...,...
12473,6g6y,KNIQVVVRCRPFNLAERKASAHSIVECDPVRKEVSVRTGSSRKTYT...,"56,90,91,92,93,94,95,96,97,98,103,104,105,106,..."
12474,3wc7,GHSYEKYNNWETIEAWTKQVTSENPDLISRTAIGTTFLGNNIYLLK...,"62,63,65,66,69,119,121,136,137,138,139,140,143..."
12475,5zhm,SMLWVGVVSIFPEMFRAISDYGITSRAVKQGLLTLTCWNPRVYTED...,"59,60,61,62,63,88,89,90,91,92,93,95,96,113,114..."
12476,3fts,VDTCSLASPASVCRTKHLHLRCSVDFTRRTLTGTAALTVQSQEDNL...,"3,130,131,132,133,134,135,146,196,262,263,266,..."


In [33]:
data_df.to_csv("binding_site/datasets/PDBbind_data.tsv", sep = "\t", index = False)